In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from childes_mi.utils.paths import DATA_DIR, FIGURE_DIR, ensure_dir
from childes_mi.utils.general import flatten,save_fig

In [3]:
from childes_mi.information_theory import model_fitting as mf

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

/home/AD/tsainbur/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [5]:
MI_DF = pd.read_pickle(DATA_DIR / "mi" / "phonbank_mi_french.pickle")

In [6]:
MI_DF[:3]

,MI,MI_var,shuff_MI,shuff_MI_var,distances,age_cohort_low,age_cohort_high,n_phones,n_unique_phones,median_n_phones,shuffling
0,"[10.663371043816312, 10.245082225102422, 10.09...","[0.004945938272197813, 0.0048637140946431075, ...","[9.97526238326807, 9.974438705111332, 9.974402...","[0.004820450882092466, 0.004817713571462185, 0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",3,20,359954,0,1626,full
1,"[10.915505015310265, 10.548361851654079, 10.39...","[0.004887922729166499, 0.004802003833059679, 0...","[10.256572952023603, 10.25706837629712, 10.257...","[0.004749013927682985, 0.004750348844609571, 0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",2.5,3,313197,0,1014,full
2,"[10.991623024066557, 10.646451779237823, 10.47...","[0.005196027243733277, 0.005117216935930317, 0...","[10.324423226440707, 10.325474461906364, 10.32...","[0.0050613373543492894, 0.005062582834832084, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",2,2.5,277162,0,724,full


In [7]:
MI_DF.shuffling.unique()

array(['full'], dtype=object)

In [8]:
MI_DF = MI_DF.assign(**{i:np.nan for i in ['exp_results', 'pow_results', 'concat_results',
     'R2_exp', 'R2_concat', 'R2_power', 'AICc_exp',
     'AICc_concat', 'AICc_power', 'bestfitmodel', 'curvature', 'min_peak']})
MI_DF['curvature'] = MI_DF['curvature'].astype(object)

In [9]:
n = 100 # max distance for computation
for idx, row in tqdm(MI_DF.iterrows(), total=len(MI_DF)):
    # get signal
    sig = np.array(row.MI-row.shuff_MI)
    distances = row.distances
    sig = sig
    
    dist_mask = distances <= row.median_n_phones
    distances = distances[dist_mask]
    sig = sig[dist_mask]
    
    # fit models
    results_power, results_exp, results_pow_exp, best_fit_model = mf.fit_models(distances, sig)
    
    # get fit results
    R2_exp, R2_concat, R2_power, AICc_exp, \
        AICc_pow, AICc_concat = mf.fit_results(sig, distances, 
                                              results_exp, results_power,
                                              results_pow_exp)
    
    
    # get model y
    distances_mod = np.logspace(0,np.log10(n), base=10, num=1000)
    if best_fit_model == 'pow_exp':
        y_model = mf.get_y(mf.pow_exp_decay, results_pow_exp, distances_mod)
    elif best_fit_model == 'exp':
        y_model = mf.get_y(mf.exp_decay, results_exp, distances_mod)
    elif best_fit_model == 'pow':
        y_model = mf.get_y(mf.powerlaw_decay, results_power, distances_mod)
    
    # get curvature of model_y
    curvature_model = mf.curvature(np.log(y_model))
    
    # if the best fit model is pow_exp, then grab the min peak
    if best_fit_model == 'pow_exp':
        # get peaks of curvature
        peaks = np.where((
            (curvature_model[:-1] < curvature_model[1:])[1:] & (curvature_model[1:] < curvature_model[:-1])[:-1]
        ))
        min_peak = peaks[0][0]
    else:
        min_peak = np.nan

    # get save model fit results to MI_DF
    MI_DF.loc[idx, np.array(['exp_results', 'pow_results', 'concat_results',
                         'R2_exp', 'R2_concat', 'R2_power', 'AICc_exp',
                         'AICc_concat', 'AICc_power', 'bestfitmodel', 'curvature', 'min_peak'])] = [
        results_exp, results_power, results_pow_exp,
        R2_exp, R2_concat, R2_power, AICc_exp,
        AICc_concat, AICc_pow, best_fit_model,
        curvature_model, min_peak
    ]

    print(row.age_cohort_low, row.age_cohort_high, best_fit_model, row.shuffling)

/mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/childes_mi/information_theory/model_fitting.py:8: RuntimeWarning: invalid value encountered in log
  return np.abs(np.log(y_true) - np.log(y_model)) * (1 / (np.log(1 + x)))
/home/AD/tsainbur/anaconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


3 20 pow_exp full
2.5 3 pow_exp full
2 2.5 pow_exp full
1.5 2 pow_exp full
1 1.5 pow_exp full
0.5 1 pow_exp full



In [10]:
MI_DF[:3]

,MI,MI_var,shuff_MI,shuff_MI_var,distances,age_cohort_low,age_cohort_high,n_phones,n_unique_phones,median_n_phones,...,concat_results,R2_exp,R2_concat,R2_power,AICc_exp,AICc_concat,AICc_power,bestfitmodel,curvature,min_peak
0,"[10.663371043816312, 10.245082225102422, 10.09...","[0.004945938272197813, 0.0048637140946431075, ...","[9.97526238326807, 9.974438705111332, 9.974402...","[0.004820450882092466, 0.004817713571462185, 0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",3,20,359954,0,1626,...,<lmfit.minimizer.MinimizerResult object at 0x7...,0.807501,0.842233,0.841002,-6284.589532,-6588.379525,-6580.378123,pow_exp,"[-5.330335864094479e-06, -8.01638653847689e-06...",169.0
1,"[10.915505015310265, 10.548361851654079, 10.39...","[0.004887922729166499, 0.004802003833059679, 0...","[10.256572952023603, 10.25706837629712, 10.257...","[0.004749013927682985, 0.004750348844609571, 0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",2.5,3,313197,0,1014,...,<lmfit.minimizer.MinimizerResult object at 0x7...,0.924930,0.956229,0.955329,-5127.795729,-5670.767263,-5654.156368,pow_exp,"[-3.437157575260229e-06, -5.172210406077421e-0...",178.0
2,"[10.991623024066557, 10.646451779237823, 10.47...","[0.005196027243733277, 0.005117216935930317, 0...","[10.324423226440707, 10.325474461906364, 10.32...","[0.0050613373543492894, 0.005062582834832084, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",2,2.5,277162,0,724,...,<lmfit.minimizer.MinimizerResult object at 0x7...,0.965343,0.985488,0.984768,-4200.066287,-4826.277413,-4795.253824,pow_exp,"[-2.0121244236145916e-07, -3.1116095472584716e...",225.0


In [11]:
MI_DF.columns

Index(['MI', 'MI_var', 'shuff_MI', 'shuff_MI_var', 'distances',
       'age_cohort_low', 'age_cohort_high', 'n_phones', 'n_unique_phones',
       'median_n_phones', 'shuffling', 'exp_results', 'pow_results',
       'concat_results', 'R2_exp', 'R2_concat', 'R2_power', 'AICc_exp',
       'AICc_concat', 'AICc_power', 'bestfitmodel', 'curvature', 'min_peak'],
      dtype='object')

In [12]:
parameter_translation = {
    "e_init": "a",
    "e_decay_const": "b",
    "p_init": "c",
    "p_decay_const": "d",
    "intercept": "f",
}

In [13]:
param_df = pd.DataFrame(
    columns=[
        "dataset", 
        "age_low", 
        "age_high", 
        "a_value", 
        "a_stderr",
        "b_value", 
        "b_stderr",
        "c_value", 
        "c_stderr",
        "d_value", 
        "d_stderr",
        "f_value", 
        "f_stderr"
    ]
)
for idx, row in MI_DF[MI_DF.shuffling == 'full'].iterrows():
    param_vals = {}
    param_stderrs = {}
    for param in row.concat_results.params:
        param_vals[param] = row.concat_results.params[param].value
        param_stderrs[param] = row.concat_results.params[param].stderr
    param_df.loc[len(param_df)] = [
        'phonbank',
        row.age_cohort_low,
        row.age_cohort_high,
        param_vals["e_init"],
        param_stderrs["e_init"],
        param_vals["e_decay_const"],
        param_stderrs["e_decay_const"],
        param_vals["p_init"],
        param_stderrs["p_init"],
        param_vals["p_decay_const"],
        param_stderrs["p_decay_const"],
        param_vals["intercept"],
        param_stderrs["intercept"],

    ]

In [14]:
param_df

,dataset,age_low,age_high,a_value,a_stderr,b_value,b_stderr,c_value,c_stderr,d_value,d_stderr,f_value,f_stderr
0,phonbank,3,20,1.031225,None,0.961794,None,0.297650,None,-1.397936,None,0.001360,None
1,phonbank,2.5,3,0.706422,0.147616,0.821490,0.070292,0.348414,0.05902,-1.310050,0.058065,0.003264,0.000054
2,phonbank,2,2.5,0.325830,0.094821,0.616843,0.060002,0.503827,0.04898,-1.434785,0.037238,0.006385,0.00006
3,phonbank,1.5,2,0.330310,0.058564,0.558299,0.035223,0.460063,0.031672,-1.242988,0.027811,0.014043,0.000124
4,phonbank,1,1.5,0.351947,0.073527,0.597750,0.040794,0.311452,0.037637,-1.086173,0.050838,0.017666,0.000315
5,phonbank,0.5,1,0.364677,0.093064,0.516317,0.050656,0.301707,0.051679,-1.153525,0.074775,0.022128,0.000411


In [15]:
ensure_dir(DATA_DIR / 'param_dfs')
param_df.to_pickle(DATA_DIR / 'param_dfs' / 'phonbank_french.pickle')